# References : for pre-processing step

<알고리즘>  
본 논문에서는 1)을 따라했다고 하며, 1)에서는 2)를 적용했다고 함.  
1) Spectral Bandits for Smooth Graph Functions (Valko et al., 2014) http://proceedings.mlr.press/v32/valko14.pdf  
2) Matrix Completion from a Few Entries (Keshavan et al., 2009) https://arxiv.org/pdf/0901.3150.pdf  
(여기 본문에는 algorithm 순서가 약간 뒤죽박죽으로 쓰인 것 같은데, 아래의 ref.code들은 결국 page 2 하단부의 Algorithm Box를 따른 듯)  

  
<코드>  
3) Matlab code(?) : https://github.com/andrewssobral/lrslibrary/tree/master/algorithms/mc/OptSpace  
4) Python code(?) : https://github.com/kkaushik7/Matrix-Completion-from-Noisy-Entries  
(여기서는 https://github.com/airoldilab/SBA/tree/master/OptSpace_matlab 의 MATLAB code를 참고했다고 하며, 3번과 다른 주소이지만 동일한 코드인 것으로 보임) 



In [1]:
!pwd

/content


In [2]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [3]:
%cd /content/gdrive

/content/gdrive


In [4]:
cd 'My Drive/Colab Notebooks/GraphUCB/RealData'

/content/gdrive/My Drive/Colab Notebooks/GraphUCB/RealData


In [5]:
import numpy as np
import pandas as pd
import sys
import math
import time
import os
import copy
from scipy import sparse
import matplotlib.pyplot as plt
import time
############
from scipy.sparse.linalg import svds
from numpy.linalg import norm
from numpy.matlib import repmat
from scipy.sparse import csr_matrix

일단은 3)을 기본 베이스로 시작 + 4) 참고


In [ ]:
def find_edges_coo(sparse_mtx):
    sparse_data = sparse_mtx.data
    sparse_row = sparse_mtx.row
    sparse_col = sparse_mtx.col
    num_row, num_col = sparse_mtx.shape

    edge_data = [1 for i in range(0, len(sparse_data))]
    E_sparse = sparse.coo_matrix((edge_data, (sparse_row, sparse_col)), shape=(num_row, num_col))

    return E_sparse

In [ ]:
class MTX_FACTORIZATION():
	def __init__(self, SPARSE_MTX, RANK, ITERATION, TOLERANCE, MAX_STEPSIZE):
		self.sparse_mtx = SPARSE_MTX
		self.row_num, self.col_num = self.sparse_mtx.shape 
		self.rank = RANK 
		self.iteration = ITERATION
		self.tolerance = TOLERANCE
		self.max_stepsize = MAX_STEPSIZE

		self.rescale_param = math.sqrt(self.sparse_mtx.nnz * self.rank / norm(self.sparse_mtx.toarray(),'fro')**2 )
  		#QQQQ : resclae_param이 살짝 다른 거 같은데... # 이건 T_r로의 projection이 아니라 진짜 그냥 rescaling인가? 
		# 코드를 돌리고 보니 주어지는 matrix의 Frobenius norm이 매우 큼. 그걸 줄이려는거 같기도 한데.
		#아니면 여기서 rescaling끝내서 projection 과정에서 상수배 안하나?
		# sparse라서 Frobenius norm 계산 시간 줄일 수 있긴 함

		#self.M_E = self.sparse_mtx.toarray()
		self.M_E = self.sparse_mtx.toarray() * self.rescale_param
  
		self.E_sparse = find_edges_coo(self.sparse_mtx)
		self.E = self.E_sparse.toarray()
		self.nonzeros = self.sparse_mtx.nnz
		self.eps = self.nonzeros / math.sqrt(self.row_num*self.col_num)

	def erase_large_degree_row_and_col(self, mtx, edge_mtx): # Algorighm의 1단계 해당
		(m, n) = mtx.shape
		tilde_mtx = copy.deepcopy(mtx) #여기서 문제가 생겼었음...! 그냥 할당시 shallow copy로 되어 여기서 tilde_mtx 만지는게 self.M_E까지 영향을 미쳤음...

		# |E|를 구하고 |E|/n 과 |E|/m을 계산 하는 대신, row, col별로 mean을 구하는 방식을 따로해서 계산했음.

		# Trim the Columns first
		edge_col_sums = np.sum(edge_mtx, axis=0)
		edge_col_mean = np.mean(edge_col_sums)
		mtx_col_sums = np.sum(mtx, axis=0)

		for col in range(0, n):
			if mtx_col_sums[col] > 2 * edge_col_mean:
				for row in range(0, m):
					tilde_mtx[row][col] = 0

		# Trim the Rows next
		edge_row_sums = np.sum(edge_mtx, axis=1)
		edge_row_mean = np.mean(edge_row_sums)
		mtx_row_sums = np.sum(mtx, axis=1)
		for row in range(0, m):
			if mtx_row_sums[row] > 2 * edge_row_mean:
				for col in range(0, n):
					tilde_mtx[row][col] = 0
		
		return tilde_mtx


	def projection_and_initialization(self, mtx_coo, rank, rows, cols, eps):
		U, Sigma, V_t = svds(mtx_coo.asfptype(), rank)
		U = U * math.sqrt(rows)
		V_t = V_t * math.sqrt(cols)
		Sigma = Sigma / eps

		return U, Sigma, V_t

	def F_t(self, X, Y_t, S, M_E, E):
		n, r = X.shape
		out = 1/2 * norm(np.multiply( np.dot(np.dot(X, S), Y_t) - M_E , E), 'fro')
		return out

	def gradF_t(self, X, Y_t, S, M_E, E):
		m, r = X.shape
		r, n = Y_t.shape

		XS = np.dot(X, S)
		YS = np.transpose(np.dot(S, Y_t))
		XSY = np.dot(XS, Y_t) ;

		Qx = np.dot(np.transpose(X), np.dot( np.multiply(M_E - XSY , E) , YS ))/ m
		Qy = np.dot(Y_t, np.dot(np.transpose(np.multiply(M_E - XSY, E)), XS)) / n

		W = np.dot( np.multiply(XSY - M_E, E), YS) + np.dot(X, Qx)
		Z = np.dot(np.transpose(np.multiply(XSY - M_E, E)) ,XS) + np.dot(np.transpose(Y_t), Qy)
		Z = np.transpose(Z)
		
		return W, Z

	def getoptS(self, X, Y_t, M_E, E):
		
		m, r = X.shape
		C = np.dot(np.dot(np.transpose(X) ,  M_E) , np.transpose(Y_t))
		col_vectors = np.zeros(r).tolist()
		for k in range(0,r):
			col_vectors[k] = np.transpose(C)[k]
		C_vec = np.concatenate(col_vectors)
		C_vec = np.transpose(C_vec)

		A = np.zeros((r**2, r**2))
  
		for i in range(0, r):
				for j in range(0, r):
						index = j*r + i
						temp = np.dot(  np.dot(np.transpose(X), np.multiply( np.outer(X[:,i],Y_t[j,:]), E )) , np.transpose(Y_t)  )
						# temp = np.dot(  np.dot(np.transpose(X),  np.outer(X[:,i],Y_t[j,:])) , np.transpose(Y_t)  )
						col_vectors = np.zeros(r).tolist()
						for k in range(0,r):
							col_vectors[k] = np.transpose(temp)[k]
						temp_vec = np.concatenate(col_vectors)
						# temp_vec = np.transpose(temp_vec)
						A[:, index] = temp_vec

		S = np.linalg.solve(A, C_vec)
		output = S.reshape((r, r))

		return output

	def getoptT(self, X, W, Y_t, Z, S, M_E, E, max_stepsize):
		norm2_W_plus_Z = norm(W,'fro')**2 + norm(Z,'fro')**2
		# print('norm2_W_plus_Z :', norm2_W_plus_Z)
		f = [0 for i in range(0, 25)]
		f[0] = self.F_t(X, Y_t, S, M_E, E)

		t = - max_stepsize ;
		current = 0
		for i in range(1, 25):
				f[i] = self.F_t( X + t*W, Y_t + t* Z, S, M_E, E)
				current = f[i]
				if  ( (f[i] - f[0]) <= 1/2 * t * math.sqrt(norm2_W_plus_Z )):
					output = t
					break
				t = t/2
		output = t
		
		#print('t가 왜 끝까지 가나: ', t, 1/2 * t * math.sqrt(norm2_W_plus_Z), current-f[0], math.sqrt(norm2_W_plus_Z) - current+f[0])
		return output

	def run(self):
		print('self.rescale_param:', self.rescale_param)
		print('shape of self.M_E:', self.M_E.shape)
		print('Frobenious norm of self.M_E:', norm(self.M_E, 'fro'))
		print('Frobenious norm of np.multiply(self.M_E, self.E):', norm(np.multiply(self.M_E, self.E), 'fro'))
		print('--------------------')
		print('Now we start the algorithm!')
		print('--------------------')
  
		# Algorithm step 1
		tilde_M_E = self.erase_large_degree_row_and_col(self.M_E, self.E)
		tilde_M_E = sparse.coo_matrix(tilde_M_E)
		print('end of step 1')

		# Algorithm step 2
		X0, S0, Y0_t = self.projection_and_initialization(tilde_M_E, self.rank, self.row_num, self.col_num, self.eps)
		print('end of step 2')
  
		# Algorithm step 3
		X = X0
		Y_t = Y0_t
		dist = np.zeros(self.iteration + 1)
		S = self.getoptS(X, Y_t, self.M_E, self.E)
  
		dist[1] = norm( np.multiply(self.M_E - np.dot(np.dot(X, S), Y_t), self.E) ,'fro') / math.sqrt(self.nonzeros)  
		print('end of step 3-1')
  
		for i in range(0, self.iteration):
			start = time.time()
		# Compute the Gradient 
			W, Z = self.gradF_t(X, Y_t, S, self.M_E, self.E)
			
		# Line search for the optimum jump length	
			t = self.getoptT(X, W, Y_t, Z, S, self.M_E, self.E, self.max_stepsize) 
			X = X + t * W
			Y_t = Y_t + t * Z
			S = self.getoptS(X, Y_t, self.M_E, self.E) 
			
		# Compute the distortion	
			dist[i+1] = norm( np.multiply(self.M_E - np.dot(np.dot(X, S), Y_t), self.E) ,'fro') / math.sqrt(self.nonzeros)  
			diff_norm = norm( self.M_E - np.dot(np.dot(X, S), Y_t) ,'fro')
			print(i,'번째 /', np.round(dist[i+1], 10) , '/ optimal_t:', t, '/ 2^{-n} :', np.log2(self.max_stepsize / -t), '/ 소요시간:', np.round(time.time()-start, 3),'초')
			print('self.M_E와 recover의 차이)) M_E = ',np.round(norm(self.M_E,'fro'),2),'전체비교시 =', np.round(diff_norm,5), '/ nnz원소들만비교시=', np.round(dist[i+1] * math.sqrt(self.nonzeros), 5))
			
			if( dist[i+1] < self.tolerance ):
				break
			
		print('end of step 3-2')
  
		return X, S, Y_t, self.rescale_param
			

# MovieLens - 1M
https://grouplens.org/datasets/movielens/1m/


All ratings are contained in the file "ratings.dat" and are in the
following format:

UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040
<font color='orange'> -> 0 to 6039 로 바꾸고 시작</font>.
- MovieIDs range between 1 and 3952
<font color='orange'> -> 0 to 3951 로 바꾸고 시작</font>.
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings

In [ ]:
#df_users = pd.read_csv('MovieLens_1M/users.dat', sep = '::', names = ['user_id, gender, age, occupation, zipcode'], engine = 'python')
#df_movies = pd.read_csv('MovieLens_1M/movies.dat', sep = '::', names = ['movie_id', 'Title', 'Genres'], engine = 'python')
df_ratings = pd.read_csv('MovieLens_1M/ratings.dat', sep = '::', names = ['user_id', 'movie_id', 'rating', 'timestamp'], engine = 'python')

In [ ]:
df_ratings.shape

(1000209, 4)

In [ ]:
df_ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
df_ratings['user_id'] = df_ratings['user_id'] - 1
df_ratings['movie_id'] = df_ratings['movie_id'] - 1
df_ratings.head()

,user_id,movie_id,rating,timestamp
0,0,1192,5,978300760
1,0,660,3,978302109
2,0,913,3,978301968
3,0,3407,4,978300275
4,0,2354,5,978824291


In [ ]:
user_range = 6039
movie_range = 3951
#user_range = df_ratings['user_id'].max()
#movie_range = df_ratings['movie_id'].max()
#print(user_range, movie_range)
df_ratings.isnull().any()

user_id      False
movie_id     False
rating       False
timestamp    False
dtype: bool

In [ ]:
user_valid = list(df_ratings['user_id'].unique())
count_na_user = 0
total_user = 0
for i in range(0, user_range + 1):
    if i in user_valid:
        total_user += 1
    else:
        count_na_user += 1
    

movie_valid = list(df_ratings['movie_id'].unique())
count_na_movie = 0
total_movie = 0
for i in range(0, movie_range + 1):
    if i in movie_valid:
        total_movie += 1
    else:
        count_na_movie += 1

print('user_range:', user_range, '/ 총 user_id 수:', total_user, '/ 없는 user_id 수:', count_na_user) 
print('movie_range:', movie_range, '/ 총 movie_id 수:', total_movie, '/ 없는 movie_id 수:', count_na_movie)

user_range: 6039 / 총 user_id 수: 6040 / 없는 user_id 수: 0
movie_range: 3951 / 총 movie_id 수: 3706 / 없는 movie_id 수: 246


In [ ]:
# movie_id를 0부터 순차적으로 되게끔 re_movie_id 라는 column을 만들자.
movie_valid.sort()
map_to_valid_movie_id =  [0 for i in range(0, movie_range+1)]
for i in range(0,len(movie_valid)):
    map_to_valid_movie_id[movie_valid[i]] = i

df_ratings['re_movie_id'] = df_ratings['movie_id'].apply(lambda x : map_to_valid_movie_id[x])

In [ ]:
mtx_elements_df = df_ratings[['user_id', 're_movie_id', 'rating']]
mtx_row_idx = list(mtx_elements_df['user_id'])
mtx_col_idx = list(mtx_elements_df['re_movie_id'])
mtx_data = list(mtx_elements_df['rating'])

In [ ]:
M0 = sparse.coo_matrix((mtx_data, (mtx_row_idx, mtx_col_idx)), shape=(total_user, total_movie))

## Start the algorithm

In [ ]:
rk = 10
iter = 100
tol = 1e-6
maxstep = 10

In [ ]:
movielens = MTX_FACTORIZATION( M0, rk , iter , tol, maxstep)
start_movielense = time.time()
final_X, final_S, final_Y_t, rescale_param = movielens.run()
print('총 소요시간 :', time.time() - start_movielense)

self.rescale_param: 0.8428839573783572
shape of self.M_E: (6040, 3706)
Frobenious norm of self.M_E: 3162.6081009193094
Frobenious norm of np.multiply(self.M_E, self.E): 3162.6081009193094
--------------------
Now we start the algorithm!
--------------------
end of step 1
end of step 2
end of step 3-1
0 번째 / 2.7674347417 / optimal_t: -0.009765625 / 2^{-n} : 10.0 / 소요시간: 21.377 초
self.M_E와 recover의 차이)) M_E =  3162.61 전체비교시 = 4508.74745 / nnz원소들만비교시= 2767.72392
1 번째 / 2.0997158065 / optimal_t: -0.00244140625 / 2^{-n} : 12.0 / 소요시간: 21.759 초
self.M_E와 recover의 차이)) M_E =  3162.61 전체비교시 = 6071.95695 / nnz원소들만비교시= 2099.93522
2 번째 / 1.5518389951 / optimal_t: -0.0006103515625 / 2^{-n} : 14.0 / 소요시간: 22.181 초
self.M_E와 recover의 차이)) M_E =  3162.61 전체비교시 = 7252.42181 / nnz원소들만비교시= 1552.00115
3 번째 / 1.3181499204 / optimal_t: -0.001220703125 / 2^{-n} : 13.0 / 소요시간: 22.12 초
self.M_E와 recover의 차이)) M_E =  3162.61 전체비교시 = 9438.25531 / nnz원소들만비교시= 1318.28766
4 번째 / 1.1846121222 / optimal_t: -0.000610

In [ ]:
# final_X, final_S, final_Y_t, rescale_param
list_true_result= [final_X, final_S / rescale_param, final_Y_t]

In [ ]:
list_true_result[0].shape

(6040, 10)

In [ ]:
# save the results as .txt or .npy file
objectname = ['X', 'S', 'Yt']
for i in range(0,3):
    # filename = 'movielens_python_optspace_'+objectname[i]+'_rank_'+str(rk)+'_iter_'+str(iter)+'_tol_'+str(tol)+'_maxstep_'+str(maxstep)+'.txt'
    # np.savetxt(filename, list_true_result[i])
    filename = 'movielens_python_optspace_'+objectname[i]+'_rank_'+str(rk)+'_iter_'+str(iter)+'_tol_'+str(tol)+'_maxstep_'+str(maxstep)+'.npy'
    np.save(filename, list_true_result[i])

In [ ]:
check = np.dot(np.dot(list_true_result[0], list_true_result[1]), list_true_result[2])
E = find_edges_coo(M0).toarray()
#edge = movielens.E
#me = movielens.M_E
print(norm(check-M0.toarray(),'fro'))
print(norm(np.multiply(check-M0.toarray(), E),'fro'))
#print(me / rescale_param -M0.toarray())

12091.594999490575
1363.5213601578616


그럼이제 알고리즘으로 얻은 결과를 사용해서 M = UV' 만들고  
U를 이용해서 RBF 그래프를 만들자 : GraphUCB Appendix K

In [ ]:
U = list_true_result[0]
V = np.transpose(np.dot(list_true_result[1], list_true_result[2]))
print(U.shape)
print(V.shape)

(6040, 10)
(3706, 10)


In [ ]:
np.save('U_user_4_sure', U)
np.save('V_user_4_sure', V)

# Netflix Prize : TBD

https://www.kaggle.com/netflix-inc/netflix-prize-data